In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

/opt/conda/envs/pennylane/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
symbols, coordinates = (['O','N', 'C', 'H', 'H', 'H', 'C'],
                        np.array([1.1280, 0.2091, 0.0000,
                                  -1.1878, 0.1791, 0.0000, 
                                  0.0598, -0.3882, 0.0000,
                                  -1.3085, 1.1864, 0.0001,
                                  -2.0305, -0.3861, -0.0001
                                -0.0014, -1.4883, -0.0001,
                                  -0.1805, 1.3955, 0.0000,
                                  
                                 ]))

sparse_matrix = qml.utils.sparse_hamiltonian(h)
eigenvalues, eigenvectors = np.linalg.eig(sparse_matrix.toarray())
theory_ground_state_energy=min(np.real(eigenvalues))
print('ground_state_energy: ', theory_ground_state_energy)

S2 = qchem.spin2(electrons,qubits, mapping='jordan_wigner')
print("Number of observables: {}\n" .format(len(h.ops)))

groups, coeffs = qml.grouping.group_observables(h.ops, h.coeffs)
print("Number of qubit-wise commuting groups:", len(groups))

for op in h.ops:
    print("Measurement {} on wires {}" .format(op.name, op.wires))
    
# dev = qml.device("default.qubit", wires=qubits)

my_bucket = f"amazon-braket-mybucketID" 
my_prefix = "my_folder_name"
s3_folder = (my_bucket, my_prefix)

device_arn = "arn:aws:braket:::device/qpu/rigetti/Aspen-9"
dev = qml.device("bracket.aws.qubit",
                 device_arn=device_arn,
                 wires=qubits,
                 s3_destination_folder=s3_folder,
                 parallel=True,
                 shots=10
                )
hf_state = qchem.hf_state(electrons, qubits)
excitations = qchem.excitations(electrons, qubits)
s_wires, d_wires = qchem.excitations_to_wires(*excitations)

def circuit(params, wires):
    qml.templates.UCCSD(params, init_state=hf_state,s_wires=s_wires, d_wires=d_wires, wires=wires)

energy_expval = qml.ExpvalCost(circuit, h, dev, optimize=True)
S2_expval = qml.ExpvalCost(circuit, S2, dev, optimize=True)

def spin(params):
    return -0.5 + np.sqrt(1/4 + S2_expval(params))

np.random.seed(1000)
params = np.random.normal(0, np.pi, len(s_wires) + len(d_wires))

opt = qml.GradientDescentOptimizer(stepsize=0.4)

epochs = 50 
energies = []
spins = []

for i in range(epochs):
    params = opt.step(energy_expval, params)
    
    e = energy_expval(params)
    s = spin(params)
    
    
    if (i + i) % 5 == 0:
        print(f"epoch: {i+1}",", Energy:", e,", Total Spin:",s)
        
    energies.append(e)
    spins.append(s)
    
    print("--------------------")
    print(f"Optimized energy: {e} Hartfree")
    print(f"Total Spin: {s}")
    
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.insert_locator import inset_axes

theory_energy = -192.0
theory_spin = 0

plt.hlines(theory_energy, 0, epochs, linestyles="dashed", colors="black")
plt.plot(energies)
plt.xlabel("Steps")
plt.ylabel("Energy Ha")

axes = plt.gca()

inset = inset_axes(axs, width="%50", height="%50", borderpad=1)
inset.hlines(theory_spin, 0, epochs, linestyles="dashed", colors="black")
inset.set_xlabel("Steps")
inset.set_ylabel("Total spin");
                       